<a href="https://colab.research.google.com/github/victoriaazabel/SEdigitalorganizations/blob/main/VictoriaZabel_assignment_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The following assignment consists again of a theoretical part (learning portfolio) and a practical part (assignment). The goal is to train a neural model for a recommendation system.

The plan would be that in the first week we will discuss your learnings from the theory part, that means you are relatively free to fill your Learning Portfolio on this topic and in the following week we will discuss your solutions of the practical part.

#Theory part (filling your Learning Portfolio, June 7)

In preparation for the practical part, I ask you to familiarize yourself with the following video sources in the next week:

1) Please watch the following videos:

https://www.youtube.com/watch?v=Fmtorg_dmM0&ab_channel=ritvikmath (not absolutely necessary, only for the overview)

https://course.fast.ai/Lessons/lesson7.html (The second part of the presentation starting with the topic collaborative filtering is mandatory)

Note: The first part of the video mainly contains tips for neural networks to submit a Kaggle Competition. For that, you would have to watch the end of the 6th video to understand this better. But this is not mandatory.

2) Please download the following notebooks and edit it in Google-Colab. Try to answer a few questions that are asked at the end. Take notes and update your Learning Portfolio.

https://www.kaggle.com/code/jhoward/collaborative-filtering-deep-dive/notebook


#Practical part (Assignment, June 14)

Find any data set that can be used for a recommender system and try to train and validate a neural network for it.

For this purpose I ask you to download a data set from the given lists and to use it for your program application.

https://gist.github.com/entaroadun/1653794

https://github.com/caserec/Datasets-for-Recommender-Systems

https://grouplens.org/datasets/movielens/

https://eigentaste.berkeley.edu/dataset/

In [1]:
# all of the usual imports

import pandas as pd
from google.colab import drive
from google.colab import data_table
import matplotlib.pyplot as plt
import torch
from torch import tensor
data_table.enable_dataframe_formatter()



In [3]:
# looking at the dataset, it's for book recommendations and split up into users, ratings, and books
# merge two csv files: user isbn ratings + book info and only keeping relevant columns

"""
drive.mount('/content/drive')
books_info = pd.read_csv('/content/drive/MyDrive/BX-Books.csv')
isbn_ratings = pd.read_csv('/content/drive/MyDrive/BX-Book-Ratings.csv')

book_reviews = isbn_ratings.merge(books_info, how='outer')
book_reviews.head(10)
"""

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


UnicodeDecodeError: ignored

In [2]:
# try two because i got "UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc3 in position 9431: invalid continuation byte" the first time

drive.mount('/content/drive')
books_info = pd.read_csv('/content/drive/MyDrive/BX-Books.csv', on_bad_lines='skip',  sep=';', encoding='latin-1')
isbn_ratings = pd.read_csv('/content/drive/MyDrive/BX-Book-Ratings.csv', on_bad_lines='skip',  sep=';', encoding='latin-1')

book_reviews = isbn_ratings.merge(books_info, on ='ISBN')
#book_reviews = isbn_ratings.merge(books_info, right_on=None)

#books_info.head(10)
#isbn_ratings.head(10)
book_reviews.head(10)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-2-8d9380ac1a1c>:4: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books_info = pd.read_csv('/content/drive/MyDrive/BX-Books.csv', on_bad_lines='skip',  sep=';', encoding='latin-1')


,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
1,2313,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
2,6543,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
3,8680,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
4,10314,034545104X,9,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
5,23768,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
6,28266,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
7,28523,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
8,39002,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
9,50403,034545104X,9,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...


In [3]:
book_reviews.shape

(1031136, 10)

In [4]:
isbn_ratings.shape

(1149780, 3)

In [5]:
books_info.shape

(271360, 8)

In [70]:
# dropping columns and re-arranging them

book_ratings = book_reviews.drop(['ISBN', 'Book-Author', 'Year-Of-Publication', 'Publisher', 'Image-URL-S', 'Image-URL-M', 'Image-URL-L'], axis=1)
book_ratings = book_ratings[['User-ID', 'Book-Title', 'Book-Rating']]
book_ratings = book_ratings.sample(frac = 0.005) # shortening because the other model took 18 hours
book_ratings.head(10)

,User-ID,Book-Title,Book-Rating
162558,184299,Hearts In Atlantis,9
814463,76352,"Who's the Junior High Hunk? (Junior High, No 9)",0
356055,87141,The Thomas Berryman Number,0
57058,79923,Unspeakable,0
221613,268711,Der Fanger Im Roggen,0
230298,188658,VÃ?Â©ronika dÃ?Â©cide de mourir,9
780527,125774,The Case of the Counterfeit Eye (Perry Mason Mysteries),0
425825,179062,Endurance: Shackleton's Incredible Voyage,0
906620,128045,Women on Men,0
708937,170742,"In Love With Her Boss (Montana Mavericks) (Silhouette Special Edition, No. 1441)",0


In [71]:
# create dataloadrs object
from fastai.collab import *
from fastai.tabular.all import *

dls = CollabDataLoaders.from_df(book_ratings, item_name = 'Book-Title', bs = 64)
dls.show_batch()

,User-ID,Book-Title,Book-Rating
0,227861,Harry Potter and the Chamber of Secrets (Book 2),0
1,257204,Witchlight,0
2,86159,Godplayer,7
3,205426,Feng Shui: Secrets That Change Your Life,0
4,158103,Dreams (Trade Size),7
5,202547,Norton Anthology of Short Fiction: Sixth Edition,10
6,271538,Empire Falls,0
7,135265,Doctor in Her Stocking: From Here to Maternity (Desire Ser. Series #1252),7
8,11676,Easy Peasy,9
9,171118,"Your Mother Was a Neanderthal (Time Warp Trio, 4)",0


In [72]:
# deeep learning model like in my learning portfolio
# embedd variables
# take results of embedding and concatenate together, recommended embedding size

embs = get_emb_sz(dls)
embs

[(2626, 131), (4625, 181)]

In [73]:
# create class with two functions

class CollabNN(Module):
    def __init__(self, user_sz, item_sz, y_range=(0,5.5), n_act=100):
        self.user_factors = Embedding(*user_sz)
        self.item_factors = Embedding(*item_sz)
        self.layers = nn.Sequential(
            nn.Linear(user_sz[1]+item_sz[1], n_act),
            nn.ReLU(),
            nn.Linear(n_act, 1))
        self.y_range = y_range

    def forward(self, x):
        embs = self.user_factors(x[:,0]),self.item_factors(x[:,1])
        x = self.layers(torch.cat(embs, dim=1))
        return sigmoid_range(x, *self.y_range)

In [74]:
# use the class to create a model
# creates embedding layers using embs sizes

model = CollabNN(*embs)

In [78]:
# learn model 1
# losses for both way too high, in training = awful model performance on the training data BUT at least it's decreasing so it's learning
# validation = way too much overfitting, seen in increase at epoch 1
# no hidden layers = 2.87 and 15.8 compared to others

learn1 = Learner(dls, model, loss_func=MSELossFlat())
learn1.fit_one_cycle(8, 0.0019, wd=0.01)    #(6, 0.0019, wd=0.001) #(6, 0.001, wd=0.01)

epoch,train_loss,valid_loss,time
0,2.884157,15.049579,00:01
1,2.896267,15.573774,00:02
2,2.992631,15.871208,00:04
3,2.959306,15.784384,00:01
4,2.908801,15.822747,00:01
5,2.910903,15.861536,00:01
6,2.881479,15.886941,00:01
7,2.879639,15.892404,00:01


In [79]:
# trying other model
# training loss is getting better but still sucks, still too much overfitting omg
# change wd (weight decay) to precent overfitting, didin't work
# dataset at 0.005 for both, final here = 0.72 and 15.7 (16 17 16 17 16...) vs. 0.7 and 16.7 (more linear decrease)

learn2 = collab_learner(dls, use_nn=True, y_range=(0, 10.5), layers=[100, 50])
learn2.fit_one_cycle(8, 0.0019, wd=0.01) #(8, 0.0019, wd=0.001)

epoch,train_loss,valid_loss,time
0,19.429995,16.778891,00:01
1,9.228504,17.756956,00:01
2,5.942934,17.871222,00:01
3,4.047812,16.652319,00:01
4,2.505490,16.217365,00:01
5,1.570121,16.069790,00:01
6,0.987572,15.929756,00:01
7,0.726681,15.792715,00:01


In [80]:
# trying other model
# training loss is getting better but still sucks, still too much overfitting omg
# change wd (weight decay) to precent overfitting, didin't work

learn3 = collab_learner(dls, use_nn=True, y_range=(0, 10.5), layers=[200, 100, 50])
learn3.fit_one_cycle(8, 0.0019, wd=0.001) #(8, 0.0019, wd=0.001)

epoch,train_loss,valid_loss,time
0,19.690643,15.275350,00:02
1,10.250325,19.012775,00:01
2,7.260207,17.357174,00:01
3,4.733693,16.812674,00:01
4,2.850390,16.711399,00:01
5,1.643178,16.651031,00:01
6,0.985350,16.668348,00:01
7,0.707007,16.722780,00:01


# Results
Model is awful, not really working out:
- The model with hidden layers are performing better than those without any
- If the data sample was > 0.005 the models performed worse
- If the data sample was > 0.01, it didn't work at all
- Increasing epochs helped
- Decreasing both the learning rate and weight decay helped
- A learning rate > 0.2 or even 0.025 the validation loss was sporadic and increased
- The weight decay had different results, sometimes 0.01 was good, now 0.001 works better. 0.0019 made it worse again
- I think the dataset is the problem
- Maybe the data is too sparce? Not enough users are writing multiple reviews? Not enough overlap between user ratings for collaborative filtering?